# Global Societal Endangerment Index (GSEI)

Development notebook

In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process
import pycountry

# Data Loading and Preprocessing
Bring datasets into the right format and perform initial transformations to form the base indicators.

Total indicator count to load: 9 + 7 + 8 + 11 + 4 = 39

In [2]:
"""
Country standardisation functions
"""

def all_countries():
    return [c.name for c in pycountry.countries]

def lookup_country(country_name: str, fuzzy_threshold=80):
    try:
        return pycountry.countries.lookup(country_name).name
    except LookupError: pass
    
    countries = all_countries()
    c_match = [c.casefold() for c in countries]
    try:
        if country_name.casefold() in c_match:
            return countries[c_match.index(country_name.casefold())]
    except Exception: pass
    
    # special cases
    if country_name == 'Canary Islands' or country_name == 'SPI':
        print(f"Using special case for '{country_name}' -> 'Spain'")
        return 'Spain'
    elif country_name == 'Turkey':
        print(f"Using special case for '{country_name}' -> 'Türkiye'")
        return 'Türkiye'
    
    # fuzzy match
    try:
        best_match, score = process.extractOne(country_name, countries)
        if score > fuzzy_threshold:
            print(f"Using fuzzy match for '{country_name}' -> '{best_match}'")
            return best_match
        else:
            raise LookupError(f"Country '{country_name}' not found (best fuzzy match too low: {best_match} ({score}))")
    except LookupError as e:
        print(e)
        return 'aaa.Unknown'

def standardise_countries(country_col: pd.Series, fuzzy_threshold=80):
    return country_col.map(lambda c: lookup_country(c, fuzzy_threshold=fuzzy_threshold))

In [3]:
def transpose_world_bank_data(wb_df, year_range):
    """
    Transpose World Bank data from long to wide format.
    """
    # clean up rows
    # drop ANT, CHI, XKX because mapping them to another country would produce duplicate country rows that are very complicated to merge
    codes_to_remove = ['ANT','CHI','XKX', 
    'AFE','AFW','ARB','CSS','CEB','EAR','EAS','EAP','TEA','EMU','ECS','ECA','TEC','EUU','FCS','HPC','HIC','IBD','IBT','IDB','IDX','IDA','LTE','LCN','LAC','TLA','LDC','LMY','LIC','LMC','MEA','MNA','TMN','MIC','NAC','INX','OED','OSS','PSS','PST','PRE','SST','SAS','TSA','SSF','SSA','TSS','UMC','WLD']
    wb_df = wb_df[~wb_df['Country Code'].isin(codes_to_remove)]
    wb_df = wb_df[~wb_df['Country Name'].str.contains("Data from", na=False)]
    wb_df = wb_df[~wb_df['Country Name'].str.contains("Last Updated", na=False)]
    wb_df = wb_df.dropna(how='all', axis=0)  # drop completely empty rows
    wb_df = wb_df.replace('..', np.nan)  # replace missing values with NaN
    # clean up columns
    wb_df['Country Name'] = standardise_countries(wb_df['Country Code'], fuzzy_threshold=100)
    wb_df = wb_df.drop(columns=['Country Code', 'Series Code'])
    wb_df.columns = [col.split(" [")[0] if "[YR" in col else col for col in wb_df.columns]
    # Pivot to wide format
    wb_df = wb_df.pivot(index='Country Name', columns='Series Name', values=[str(year) for year in year_range])
    wb_df.columns = ['_'.join(col).strip() for col in wb_df.columns.values]
    wb_df = wb_df.reset_index()
    return wb_df

def indicator_yearly_availability(processed_wb_df):
    """
    Calculate the availability of data for each indicator and year.
    """
    # Select indicator columns
    indicator_cols = processed_wb_df.columns[1:]
    # Calculate availability (non-null percentage)
    availability = processed_wb_df[indicator_cols].notnull().mean()
    # Reshape to DataFrame
    availability_df = availability.to_frame(name='value').reset_index()
    availability_df.rename(columns={'index': 'indicator'}, inplace=True)
    # Split indicator column
    availability_df[['year', 'indicatorName']] = availability_df['indicator'].str.split('_', n=1, expand=True)
    # Drop original indicator column
    availability_df.drop('indicator', axis=1, inplace=True)
    # Pivot to desired format
    return availability_df.pivot(index='indicatorName', columns='year', values='value').reset_index()

### Environmental Risks


In [4]:
# start with comprehensive country list
env_data = pd.DataFrame(all_countries(), columns=['Country'])

env_data

,Country
0,Aruba
1,Afghanistan
2,Angola
3,Anguilla
4,Åland Islands
...,...
244,Samoa
245,Yemen
246,South Africa
247,Zambia


In [14]:
# Climate Change Vulnerability

# Maximum relative temperature change (°C) in the last 10 years, compared to a 1951-1980 baseline
temp_change = pd.read_csv("data/UN_FAO_climate_change_indicators.csv")
# drop row with Country 'world' in temp_change
temp_change = temp_change[temp_change['Country'] != 'World']
# standardise country names
temp_change['Country'] = standardise_countries(temp_change['ISO3'])
# INDICATOR: calculate maximum value between F2012 and F2022
indicator_cols = ['Temp Change max 2012-2022']
temp_change[indicator_cols[0]] = temp_change[['F2012', 'F2013', 'F2014', 'F2015', 'F2016', 'F2017', 'F2018', 'F2019', 'F2020', 'F2021', 'F2022']].max(axis=1)
# merge
if indicator_cols[0] in env_data.columns:
    env_data = env_data.drop(columns=indicator_cols)
env_data = env_data.merge(temp_change[['Country'] + indicator_cols], on='Country', how='left')

print(env_data.columns.values)

['Country' 'Disaster Affected Population 2020-2025'
 'Disaster Damage US$ 2020-2025' 'Temp Change max 2012-2022']


In [15]:
# Exposure to Natural Disasters (earthquakes, floods, hurricanes)

disasters = pd.read_excel("data/EM-DAT_natural_disasters_ALL_2020-2025.xlsx")
# standardise country names
disasters['Country'] = standardise_countries(disasters['ISO'])
# group by country, summing up INDICATORS: affected people + total damage
disasters_grouped = disasters.groupby('Country')[['No. Affected', 'Total Damage, Adjusted (\'000 US$)']].sum().reset_index()
disasters_grouped['Total Damage, Adjusted (\'000 US$)'] = disasters_grouped['Total Damage, Adjusted (\'000 US$)'] * 1000
# rename columns
indicator_cols = ['Disaster Affected Population 2020-2025', 'Disaster Damage US$ 2020-2025']
disasters_grouped.columns = ['Country'] + indicator_cols
# merge
if indicator_cols[0] in env_data.columns:
    env_data = env_data.drop(columns=indicator_cols)
env_data = env_data.merge(disasters_grouped, on='Country', how='left')

print(env_data.columns.values)

Using special case for 'SPI' -> 'Spain'
Using special case for 'SPI' -> 'Spain'
['Country' 'Temp Change max 2012-2022'
 'Disaster Affected Population 2020-2025' 'Disaster Damage US$ 2020-2025']


In [129]:
# Air and Water Pollution Levels

# INDICATORS:
# Years of lost life due to unsafe water, sanitation, and handwashing
# Years of lost life due to air pollution

air_water = pd.read_csv("data/IHME_GBD_environmental_risk_export_2021.csv")
# standardise country names
air_water['Location'] = standardise_countries(air_water['Location'], fuzzy_threshold=79)
# rename columns
indicator_cols = ['Unsafe water, sanitation (YLL)', 'Air pollution (YLL)']
air_water.columns = ['Country'] + indicator_cols
# merge
if indicator_cols[0] in env_data.columns:
    env_data = env_data.drop(columns=indicator_cols)
env_data = env_data.merge(air_water, on='Country', how='left')

print(env_data.columns.values)

Using fuzzy match for 'Cape Verde' -> 'Cabo Verde'
Using fuzzy match for 'Cote d'Ivoire' -> 'Côte d'Ivoire'
Using fuzzy match for 'Democratic Republic of Congo' -> 'Congo, The Democratic Republic of the'
Using fuzzy match for 'Iran (Islamic Republic of)' -> 'Iran, Islamic Republic of'
Using fuzzy match for 'Libyan Arab Jamahiriya' -> 'Libya'
Using fuzzy match for 'Republic of Congo' -> 'Congo'
Using fuzzy match for 'Republic of Korea' -> 'Korea, Republic of'
Using special case for 'Turkey' -> 'Türkiye'
['Country' 'Temp Change max 2012-2022'
 'Disaster Affected Population 2020-2025' 'Disaster Damage US$ 2020-2025'
 'Unsafe water, sanitation (YLL)' 'Air pollution (YLL)']


In [130]:
wb_dev2 = pd.read_csv("data/Worldbank_development_2.csv")
wb_dev2 = transpose_world_bank_data(wb_dev2, range(2014, 2024))

wb_dev2

,Country Name,2014_Access to electricity (% of population),"2014_External debt stocks, total (DOD, current US$)",2014_Individuals using the Internet (% of population),"2014_Inflation, consumer prices (annual %)",2014_People using safely managed drinking water services (% of population),2014_Prevalence of moderate or severe food insecurity in the population (%),2014_Prevalence of undernourishment (% of population),2014_Renewable energy consumption (% of total final energy consumption),2014_Renewable internal freshwater resources per capita (cubic meters),...,"2023_Inflation, consumer prices (annual %)",2023_People using safely managed drinking water services (% of population),2023_Prevalence of moderate or severe food insecurity in the population (%),2023_Prevalence of undernourishment (% of population),2023_Renewable energy consumption (% of total final energy consumption),2023_Renewable internal freshwater resources per capita (cubic meters),2023_Secure Internet servers (per 1 million people),2023_Total reserves (% of total external debt),"2023_Unemployment, total (% of total labor force) (modeled ILO estimate)","2023_Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)"
0,Afghanistan,89.5,2529865267.8,7,4.67399603536305,22.9443007139289,NaN,19.3,19.1,1437.82776335935,...,NaN,NaN,NaN,NaN,NaN,NaN,43.758544404586,NaN,13.991,17.291
1,Albania,100,8512452310,54.3,1.6258650440261,70.5632610188294,NaN,4.5,38.6,9310.84516168334,...,4.75976421930107,NaN,NaN,NaN,NaN,NaN,1220.33290943972,56.806329270167,10.108,24.817
2,Algeria,99.3,5521188948.6,29.5,2.91692692067458,76.6182319353339,NaN,2.7,0.1,286.876447055991,...,9.32217375928322,NaN,NaN,NaN,NaN,NaN,100.294126063305,1110.23630387233,11.701,30.447
3,American Samoa,NaN,NaN,NaN,NaN,89.165529706952,NaN,NaN,0.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,462.953220681383,NaN,NaN,NaN
4,Andorra,100,NaN,86.1,NaN,90.6400003669046,NaN,NaN,19.4,4280.07648805891,...,NaN,NaN,NaN,NaN,NaN,NaN,14618.5811813595,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,"Virgin Islands, British",99.2,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1982403.48852123,NaN,NaN,NaN
211,"Virgin Islands, U.S.",100,NaN,50.07,NaN,97.9360043349244,NaN,NaN,4.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,305.003002373305,NaN,12.32,25.824
212,Yemen,66.1,7723142216.8,22.55,8.10472583623915,NaN,NaN,32.4,0.8,69.4758992902508,...,NaN,NaN,NaN,NaN,NaN,NaN,6.95593912679964,NaN,17.091,32.395
213,Zambia,27.9,9675902727,6.5,7.80687553566333,NaN,NaN,36,84.6,5045.51183792205,...,10.884531691282,NaN,NaN,NaN,NaN,NaN,54.9605251697733,NaN,5.905,9.752


In [131]:
# check availability of data for each indicator over the years
indicator_yearly_availability(wb_dev2)

year,indicatorName,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Access to electricity (% of population),0.995349,0.995349,0.995349,0.995349,0.995349,0.995349,0.995349,0.995349,0.995349,0.000000
1,"External debt stocks, total (DOD, current US$)",0.544186,0.553488,0.553488,0.553488,0.553488,0.553488,0.553488,0.553488,0.553488,0.553488
2,Individuals using the Internet (% of population),0.934884,0.930233,0.944186,0.953488,0.813953,0.865116,0.893023,0.883721,0.851163,0.274419
3,"Inflation, consumer prices (annual %)",0.874419,0.869767,0.869767,0.846512,0.832558,0.832558,0.804651,0.804651,0.800000,0.758140
4,People using safely managed drinking water ser...,0.637209,0.637209,0.641860,0.641860,0.637209,0.637209,0.637209,0.627907,0.613953,0.000000
5,Prevalence of moderate or severe food insecuri...,0.000000,0.455814,0.460465,0.539535,0.572093,0.627907,0.683721,0.706977,0.702326,0.000000
6,Prevalence of undernourishment (% of population),0.795349,0.795349,0.795349,0.795349,0.795349,0.790698,0.795349,0.795349,0.795349,0.000000
7,Renewable energy consumption (% of total final...,0.986047,0.986047,0.986047,0.986047,0.986047,0.986047,0.986047,0.986047,0.330233,0.000000
8,Renewable internal freshwater resources per ca...,0.851163,0.851163,0.851163,0.851163,0.851163,0.851163,0.851163,0.851163,0.000000,0.000000
9,Secure Internet servers (per 1 million people),0.976744,0.986047,0.995349,0.995349,0.990698,0.986047,0.995349,1.000000,1.000000,1.000000


#### Year selection for Worldbank Development Indicators (dataset 2)

Based on the availability of data for the Worldbank Development Indicators, one year is selected for each indicator that balances recency and data availability. The missing data will be handled in the imputation step. The selected years are:

- 2022 (99%) <- Access to electricity (% of population)
- 2023 (55%) <- "External debt stocks, total (DOD, current US$)"
- 2022 (85%) <- Individuals using the Internet (% of population)
- 2023 (75%) <- "Inflation, consumer prices (annual %)"
- 2022 (61%) <- People using safely managed drinking water services (% of population)
- 2022 (70%) <- Prevalence of moderate or severe food insecurity in the population (%)
- 2022 (80%) <- Prevalence of undernourishment (% of population)
- 2021 (98%) <- Renewable energy consumption (% of total final energy consumption)
- 2021 (85%) <- Renewable internal freshwater resources per capita (cubic meters)
- 2023 (100%) <- Secure Internet servers (per 1 million people)
- 2021 (44%) <- Total reserves (% of total external debt)
- 2023 (85%) <- "Unemployment, total (% of total labor force) (modeled ILO estimate)"
- 2023 (85%) <- "Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)"


In [132]:
# Water Scarcity & Food Security

# INDICATORS:
# Renewable internal freshwater resources per capita (cubic meters)
# People using safely managed drinking water services (% of population)
# Prevalence of moderate or severe food insecurity in the population (%)
# Prevalence of undernourishment (% of population)

water_food = wb_dev2[['2022_People using safely managed drinking water services (% of population)', '2022_Prevalence of moderate or severe food insecurity in the population (%)', '2022_Prevalence of undernourishment (% of population)', '2021_Renewable internal freshwater resources per capita (cubic meters)']]
# convert to numeric TODO move to wb init
water_food = water_food.apply(pd.to_numeric, errors='coerce')
water_food = pd.concat([wb_dev2['Country Name'], water_food], axis=1)
# rename columns
indicator_cols = ['Safe Drinking Water (%)', 'Food Insecurity (%)', 'Undernourishment (%)', 'Renewable Freshwater per Capita (m3)']
water_food.columns = ['Country'] + indicator_cols
# merge
if indicator_cols[0] in env_data.columns:
    env_data = env_data.drop(columns=indicator_cols)
env_data = env_data.merge(water_food, on='Country', how='left')

print(env_data.columns.values)

['Country' 'Temp Change max 2012-2022'
 'Disaster Affected Population 2020-2025' 'Disaster Damage US$ 2020-2025'
 'Unsafe water, sanitation (YLL)' 'Air pollution (YLL)'
 'Safe Drinking Water (%)' 'Food Insecurity (%)' 'Undernourishment (%)'
 'Renewable Freshwater per Capita (m3)']


### Political Instability & Governance


In [17]:
# Governance Quality

wb_gov = pd.read_csv('data/Worldbank_governance.csv')
wb_gov = transpose_world_bank_data(wb_gov, range(2014, 2024))

indicator_yearly_availability(wb_gov)

year,indicatorName,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Control of Corruption: Estimate,0.981132,0.990566,0.990566,0.990566,0.990566,0.990566,0.990566,0.990566,1.000000,1.000000
1,Government Effectiveness: Estimate,0.981132,0.990566,0.990566,0.990566,0.990566,0.990566,0.990566,0.990566,1.000000,1.000000
2,Political Stability and Absence of Violence/Te...,0.990566,0.990566,0.990566,0.990566,1.000000,1.000000,1.000000,1.000000,1.000000,0.995283
3,Regulatory Quality: Estimate,0.981132,0.990566,0.990566,0.990566,0.990566,0.990566,0.990566,0.990566,1.000000,1.000000
4,Rule of Law: Estimate,0.981132,0.990566,0.990566,0.990566,0.990566,0.990566,0.990566,0.990566,1.000000,1.000000
5,Voice and Accountability: Estimate,0.957547,0.957547,0.957547,0.957547,0.971698,0.976415,0.976415,0.976415,0.976415,0.962264


In [18]:
# use 2023 since all data is >95% available

# INDICATORS:
# Corruption Control
# Rule of Law
# Political Stability
# Government Effectiveness
# Regulatory Quality
# Voice and Accountability

wb_gov

,Country Name,2014_Control of Corruption: Estimate,2014_Government Effectiveness: Estimate,2014_Political Stability and Absence of Violence/Terrorism: Estimate,2014_Regulatory Quality: Estimate,2014_Rule of Law: Estimate,2014_Voice and Accountability: Estimate,2015_Control of Corruption: Estimate,2015_Government Effectiveness: Estimate,2015_Political Stability and Absence of Violence/Terrorism: Estimate,...,2022_Political Stability and Absence of Violence/Terrorism: Estimate,2022_Regulatory Quality: Estimate,2022_Rule of Law: Estimate,2022_Voice and Accountability: Estimate,2023_Control of Corruption: Estimate,2023_Government Effectiveness: Estimate,2023_Political Stability and Absence of Violence/Terrorism: Estimate,2023_Regulatory Quality: Estimate,2023_Rule of Law: Estimate,2023_Voice and Accountability: Estimate
0,Afghanistan,-1.36493408679962,-1.36164617538452,-2.41106843948364,-1.12413370609283,-1.4377019405365,-1.13552260398865,-1.35471308231354,-1.39739489555359,-2.56262516975403,...,-2.54495596885681,-1.27180624008179,-1.65984559059143,-1.75352871417999,-1.15493178367615,-1.98701369762421,-2.48408055305481,-1.26642525196075,-1.64964091777802,-1.85290133953094
1,Albania,-0.5865718126297,-0.0497708097100258,0.485986232757568,0.279037684202194,-0.311199098825455,0.143689781427383,-0.54514080286026,0.0275752544403076,0.341639041900635,...,0.106261312961578,0.159353822469711,-0.165585100650787,0.141740188002586,-0.332218527793884,0.250855356454849,0.183348968625069,0.171953663229942,-0.164097368717194,0.168997228145599
2,Algeria,-0.61311799287796,-0.339864790439606,-1.19053518772125,-1.30188941955566,-0.804650962352753,-0.813460826873779,-0.639462769031525,-0.410829484462738,-1.09078657627106,...,-0.650652348995209,-1.06357324123383,-0.833259999752045,-1.01191890239716,-0.589308321475983,-0.670135140419006,-0.577885270118713,-0.948087155818939,-0.678364396095276,-0.979883790016174
3,American Samoa,1.2206939458847,0.438561886548996,1.08068346977234,0.177999645471573,1.40731287002563,NaN,1.15172910690308,0.442817568778992,1.15167808532715,...,1.12317395210266,0.545899748802185,1.22111594676971,0.952084183692932,1.25135612487793,0.654167771339417,1.11422121524811,0.512013554573059,1.21249115467072,NaN
4,Andorra,1.2206939458847,1.71216595172882,1.28659331798553,0.880748271942139,1.63860523700714,1.16595566272736,1.15172910690308,1.74073541164398,1.36598539352417,...,1.58598840236664,1.39833402633667,1.48548305034637,1.10678911209106,1.25135612487793,1.4754387140274,1.58346593379974,1.35054576396942,1.47689890861511,0.996939778327942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,Viet Nam,-0.438487648963928,-0.0470734238624573,-0.0223473571240902,-0.602058231830597,-0.360228449106216,-1.37489593029022,-0.457950294017792,0.0631621554493904,0.0638082250952721,...,-0.0461320951581001,-0.429956316947937,-0.158469453454018,-1.28640735149384,-0.415814280509949,0.126466900110245,-0.0363807342946529,-0.382607728242874,-0.0854718759655952,-1.24185371398926
208,"Virgin Islands, U.S.",0.634795427322388,1.20272433757782,1.16425490379333,0.529373943805695,0.0195597112178802,NaN,0.571842968463898,1.22156822681427,1.22000181674957,...,0.529783725738525,1.39833402633667,0.956748783588409,NaN,-0.018456656485796,0.654167771339417,0.534723103046417,1.35054576396942,0.948083341121674,NaN
209,Yemen,-1.56582367420197,-1.43657875061035,-2.66557502746582,-0.909684360027313,-1.19622910022736,-1.30112385749817,-1.50848495960236,-1.71452248096466,-2.66801619529724,...,-2.462735414505,-1.91754972934723,-1.85193991661072,-1.61970317363739,-1.64893329143524,-2.27542042732239,-2.56293869018555,-1.8430507183075,-1.83897626399994,-1.55021691322327
210,Zambia,-0.34807288646698,-0.534005224704742,0.156212016940117,-0.533904850482941,-0.22469437122345,-0.109524846076965,-0.368724584579468,-0.643124341964722,0.144709780812263,...,0.161331877112389,-0.529057919979095,-0.517023503780365,-0.0794507637619972,-0.477706879377

In [134]:
# Regime Type


### Social Vulnerability


### Economic Instability & Infrastructure


### Global & Regional Threats


In [135]:
env_data

,Country,Temp Change max 2012-2022,Disaster Affected Population 2020-2025,Disaster Damage US$ 2020-2025,"Unsafe water, sanitation (YLL)",Air pollution (YLL),Safe Drinking Water (%),Food Insecurity (%),Undernourishment (%),Renewable Freshwater per Capita (m3)
0,Aruba,1.303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,2.012,13016058.0,0.000000e+00,1422.29,4283.98,30.034098,80.9,30.4,1178.737859
2,Angola,1.752,4544311.0,0.000000e+00,1710.48,2429.79,NaN,79.2,23.2,4285.826520
3,Anguilla,1.224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Åland Islands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
244,Samoa,1.440,NaN,NaN,109.99,3146.24,62.191714,23.6,5.4,NaN
245,Yemen,NaN,1885344.0,2.354600e+07,471.74,2859.66,NaN,72.5,39.5,56.542461
246,South Africa,1.811,12286813.0,3.910613e+09,1360.48,1806.03,NaN,19.4,8.1,728.424454
247,Zambia,1.450,12283962.0,0.000000e+00,1903.05,3103.66,NaN,NaN,35.4,4091.083850
